# Part 2: Store Enhanced Data in a Database

Indego publishes quarterly data on city bike usage. In Part 1, this data was cleaned and enhanced with additional information, to prepare it for analysis. In Part 2, this data will be stored into a SQLite database, so it can be accessed in the future using SQL and without needing to read the full dataset into memory

Jump to a specific section:
<br> *(jump to does not work if viewing thru GitHub)*<br>
- [Load data into dataframes](#Load-data-into-dataframes)
- [Restructure data to fit a relational database](#Restructure-data-to-fit-a-relational-database)
- [Store data as a SQL database](#Store-data-as-a-SQL-database)

<br><br>
## Load data into dataframes

[Return to Top](#Part-2:-Store-Enhanced-Data-in-a-Database)

Data on Indego stations is stored in a single csv file

In [3]:
import os
import pandas as pd
import numpy as np

# Load from csv in the current working director
station_data = pd.read_csv(os.getcwd() + "/staged_data/indego-stations-2021-10-01-enhanced.csv")
tsd_data = pd.read_csv(os.getcwd() + "/staged_data/indego-trips-enhanced.csv")
station_name_combos = pd.read_csv(os.getcwd() + "/staged_data/stations-with-distance.csv", dtype={'bike_id':'object'})

<br><br>
## Restructure data to fit a relational database
[Return to Top](#Part-2:-Store-Enhanced-Data-in-a-Database)

<br> Prepare the data to be stored in a relational database, starting with station data

In [4]:
station_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   station_id           180 non-null    int64 
 1   station_name         180 non-null    object
 2   day_of_go_live_date  179 non-null    object
 3   status               180 non-null    object
 4   neighborhood         180 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.2+ KB


<br>Add latitude and longitude coordinates to station data, where available. This information is currently stored for each individual trip, rather than once per station.

In [5]:
# Get the list of all stations for start or end of trip, with latlong data. Avg latlong if multiple entries exist
start_coords = tsd_data[["start_station", "start_lat", "start_lon"]].groupby(["start_station"]).mean().reset_index()
end_coords = tsd_data[["end_station", "end_lat", "end_lon"]].groupby(["end_station"]).mean().reset_index()

# Rename columns to match
start_coords = start_coords.rename(columns={"start_station":"station_id", "start_lat":"station_lat", "start_lon":"station_lon"})
end_coords = end_coords.rename(columns={"end_station":"station_id", "end_lat":"station_lat", "end_lon":"station_lon"})

# Merge the two lists. Avg latlong if multple entries exist
coords = start_coords.append(end_coords).groupby("station_id").mean().reset_index()

station_data_sql = pd.merge(left=station_data, right=coords[["station_id", "station_lat", "station_lon"]], how="left", left_on=["station_id"], right_on=["station_id"])

station_data_sql.head(5)

,station_id,station_name,day_of_go_live_date,status,neighborhood,station_lat,station_lon
0,3000,Virtual Station,4/23/2015,Active,Center City,NaN,NaN
1,3004,Municipal Services Building Plaza,4/23/2015,Active,Center City,39.953781,-75.163742
2,3005,"Welcome Park, NPS",4/23/2015,Active,Center City East,39.947330,-75.144028
3,3006,40th & Spruce,4/23/2015,Active,University City,39.952202,-75.203110
4,3007,"11th & Pine, Kahn Park",4/23/2015,Active,Washington Square West,39.945171,-75.159927


<br>Check if any station IDs are missing longlat data. Only stations that are inactive and the virtual station do not have data

In [6]:
station_data_sql.loc[station_data_sql["station_lat"].isnull() == True]

,station_id,station_name,day_of_go_live_date,status,neighborhood,station_lat,station_lon
0,3000,Virtual Station,4/23/2015,Active,Center City,NaN,NaN
20,3023,Rittenhouse Square,4/23/2015,Inactive,Rittenhouse Square,NaN,NaN
43,3048,Broad & Fitzwater,4/23/2015,Inactive,South Philadelphia,NaN,NaN
90,3109,Parkside & Girard,5/6/2016,Inactive,East Parkside,NaN,NaN
103,3122,"24th & Cecil B. Moore, Cecil B. Moore Library",4/27/2016,Inactive,North Philadelphia,NaN,NaN


<br>Convert the go live date to a datetime before storing in the SQL db

In [9]:
import datetime as dt

station_data_sql["day_of_go_live_date"] = pd.to_datetime(station_data_sql["day_of_go_live_date"])
station_data_sql = station_data_sql.rename(columns={"day_of_go_live_date": "go_live_date"})

station_data_sql.head(2)

,station_id,station_name,go_live_date,status,neighborhood,station_lat,station_lon
0,3000,Virtual Station,2015-04-23,Active,Center City,NaN,NaN
1,3004,Municipal Services Building Plaza,2015-04-23,Active,Center City,39.953781,-75.163742


<br>Store the combination of start & end stations with distance in between as its own table

In [10]:
station_name_combos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23940 entries, 0 to 23939
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   start_name  23940 non-null  object 
 1   end_name    23940 non-null  object 
 2   start_lat   23760 non-null  float64
 3   start_lon   23760 non-null  float64
 4   end_lat     23760 non-null  float64
 5   end_lon     23760 non-null  float64
 6   count       23760 non-null  float64
 7   distance    23940 non-null  object 
dtypes: float64(5), object(3)
memory usage: 1.5+ MB


<br>Drop the latlong data from this table - it is now stored in the station data table

In [42]:
station_combos_sql = station_name_combos.copy()

station_combos_sql = station_combos_sql.drop(columns=["start_lat", "start_lon", "end_lat", "end_lon", "count"])

station_combos_sql.head()

,start_name,end_name,distance
0,10th & Chestnut,10th & Federal,1.1 mi
1,10th & Chestnut,11th & Market,0.4 mi
2,10th & Chestnut,"11th & Pine, Kahn Park",0.6 mi
3,10th & Chestnut,"11th & Poplar, John F. Street Community Center",1.6 mi
4,10th & Chestnut,11th & Reed,1.3 mi


<br>Replace the start name and end name with respective stations IDs - station ID will used consistently across the schema as the key to join tables

In [43]:
# Join start station data
station_combos_sql = pd.merge(left=station_combos_sql, right=station_data_sql[["station_id", "station_name"]], how="left", left_on="start_name", right_on="station_name").drop(columns=["station_name"])
station_combos_sql = station_combos_sql.rename(columns={"station_id":"start_id"})

# Join end station data
station_combos_sql = pd.merge(left=station_combos_sql, right=station_data_sql[["station_id", "station_name"]], how="left", left_on="end_name", right_on="station_name").drop(columns=["station_name"])
station_combos_sql = station_combos_sql.rename(columns={"station_id":"end_id"})

# Drop the start name and end name columns
station_combos_sql = station_combos_sql.drop(columns=["start_name", "end_name"])

# Reorder the columns
station_combos_sql = station_combos_sql[["start_id", "end_id", "distance"]]

station_combos_sql.head(5)

,start_id,end_id,distance
0,3033,3098,1.1 mi
1,3033,3185,0.4 mi
2,3033,3007,0.6 mi
3,3033,3016,1.6 mi
4,3033,3034,1.3 mi


In [44]:
station_combos_sql.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24518 entries, 0 to 24517
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   start_id  24518 non-null  int64 
 1   end_id    24518 non-null  int64 
 2   distance  24518 non-null  object
dtypes: int64(2), object(1)
memory usage: 766.2+ KB


<br>Store trip data as its own table

<br>Note: Since this database will be used primarily for querying and not storing transactions on an ongoing basis, distance will be stored in this data as well as in the station_combos table. Having this data readily available for querying will save time in analysis - distance data will not need to be joined dynamically each time it is needed

In [45]:
tsd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990500 entries, 0 to 1990499
Data columns (total 20 columns):
 #   Column               Dtype  
---  ------               -----  
 0   trip_id              int64  
 1   duration             int64  
 2   start_time           object 
 3   end_time             object 
 4   start_station        int64  
 5   start_lat            float64
 6   start_lon            float64
 7   end_station          int64  
 8   end_lat              float64
 9   end_lon              float64
 10  bike_id              object 
 11  plan_duration        float64
 12  trip_route_category  object 
 13  passholder_type      object 
 14  bike_type            object 
 15  start_name           object 
 16  start_neighborhood   object 
 17  end_name             object 
 18  end_neighborhood     object 
 19  distance             float64
dtypes: float64(6), int64(4), object(10)
memory usage: 303.7+ MB


<br>Drop the data from this table that is already stored in station data table 

<br>Note: Since this database will be used primarily for querying and not storing transactions on an ongoing basis, distance will be stored in this data as well as in the station_combos table. Having this data readily available for querying will save time in analysis - distance data will not need to be joined dynamically each time it is needed

In [86]:
trip_data_sql = tsd_data.copy()

trip_data_sql = tsd_data.drop(columns=["start_lat", "start_lon", "end_lat", "end_lon", "start_name", "start_neighborhood", "end_name", "end_neighborhood"])

trip_data_sql.tail()

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,distance
1990495,428365176,10,9/30/2021 23:57,10/1/2021 0:07,3009,3035,18791,30.0,One Way,Indego30,electric,0.8
1990496,428365174,7,9/30/2021 23:57,10/1/2021 0:04,3047,3028,5263,30.0,One Way,Indego30,standard,0.8
1990497,428365172,7,9/30/2021 23:58,10/1/2021 0:05,3046,3050,18675,30.0,One Way,Indego30,electric,0.7
1990498,428365170,3,9/30/2021 23:58,10/1/2021 0:01,3115,3075,21618,30.0,One Way,Indego30,electric,0.5
1990499,428365168,12,9/30/2021 23:59,10/1/2021 0:11,3012,3112,18807,30.0,One Way,Indego30,electric,3.0


<br>The bike_id column is storing as an object, rather than int - indicating non-integer values. See below

In [87]:
trip_data_sql.loc[trip_data_sql['bike_id'].str.isdigit() ==  False]

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,distance
1239893,356150624,79,9/28/2020 18:54,9/28/2020 20:13,3159,3206,46RUSSELL,30.0,One Way,Indego30,standard,2.0
1338199,361127740,54,11/10/2020 19:13,11/10/2020 20:07,3204,3208,47ERIN,365.0,One Way,Indego365,standard,2.7
1443502,368796659,128,3/2/2021 14:16,3/2/2021 16:24,3209,3010,WAND_ERIN_47,30.0,One Way,Indego30,standard,3.1
1638469,391010723,48,6/6/2021 14:53,6/6/2021 15:41,3206,3211,WAND_LILRUSS_43,1.0,One Way,Day Pass,standard,1.6
1794797,407706851,6,7/31/2021 16:14,7/31/2021 16:20,3108,3101,WAND_PHIL_61,1.0,One Way,Day Pass,standard,1.1


<br>Drop the 5 rows with invalid bike IDs, then reformat the column to store as an int

In [88]:
trip_data_sql = trip_data_sql.loc[trip_data_sql['bike_id'].str.isdigit() !=  False]
trip_data_sql['bike_id'] = trip_data_sql['bike_id'].astype('int64') 
trip_data_sql.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990495 entries, 0 to 1990499
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   trip_id              int64  
 1   duration             int64  
 2   start_time           object 
 3   end_time             object 
 4   start_station        int64  
 5   end_station          int64  
 6   bike_id              int64  
 7   plan_duration        float64
 8   trip_route_category  object 
 9   passholder_type      object 
 10  bike_type            object 
 11  distance             float64
dtypes: float64(2), int64(5), object(5)
memory usage: 197.4+ MB


<br>Convert the date columns to datetimes before storing in the SQL db

In [90]:
trip_data_sql["start_time"] = pd.to_datetime(trip_data_sql["start_time"])
trip_data_sql["end_time"] = pd.to_datetime(trip_data_sql["end_time"])

trip_data_sql.tail()

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,distance
1990495,428365176,10,2021-09-30 23:57:00,2021-10-01 00:07:00,3009,3035,18791,30.0,One Way,Indego30,electric,0.8
1990496,428365174,7,2021-09-30 23:57:00,2021-10-01 00:04:00,3047,3028,5263,30.0,One Way,Indego30,standard,0.8
1990497,428365172,7,2021-09-30 23:58:00,2021-10-01 00:05:00,3046,3050,18675,30.0,One Way,Indego30,electric,0.7
1990498,428365170,3,2021-09-30 23:58:00,2021-10-01 00:01:00,3115,3075,21618,30.0,One Way,Indego30,electric,0.5
1990499,428365168,12,2021-09-30 23:59:00,2021-10-01 00:11:00,3012,3112,18807,30.0,One Way,Indego30,electric,3.0


<br><br>
## Store data as a SQL database
[Return to Top](#Part-2:-Store-Enhanced-Data-in-a-Database)

<br>By storing the data this way, it can be queried in the future using SQL, which will not require the full dataset to be loaded into memory and allows for faster query execution

<br>Create a new SQLite database

In [91]:
import sqlite3

conn = sqlite3.connect('bike_trips.db')
c = conn.cursor()

In [92]:
station_data_sql.to_sql('stations', conn, if_exists='replace', index = False)
station_combos_sql.to_sql('station_combos', conn, if_exists='replace', index = False)
trip_data_sql.to_sql('trips', conn, if_exists='replace', index = False)

<br>Check header info

In [93]:
c.execute('''  
            PRAGMA table_info(stations);
            ''')

for row in c.fetchall():
    print (row)

(0, 'station_id', 'INTEGER', 0, None, 0)
(1, 'station_name', 'TEXT', 0, None, 0)
(2, 'go_live_date', 'TIMESTAMP', 0, None, 0)
(3, 'status', 'TEXT', 0, None, 0)
(4, 'neighborhood', 'TEXT', 0, None, 0)
(5, 'station_lat', 'REAL', 0, None, 0)
(6, 'station_lon', 'REAL', 0, None, 0)


<br>Check data

In [94]:
c.execute('''  
            SELECT * 
            FROM stations
            LIMIT 10;
            ''')

for row in c.fetchall():
    print (row)

(3000, 'Virtual Station', '2015-04-23 00:00:00', 'Active', 'Center City', None, None)
(3004, 'Municipal Services Building Plaza', '2015-04-23 00:00:00', 'Active', 'Center City', 39.953781, -75.163742)
(3005, 'Welcome Park, NPS', '2015-04-23 00:00:00', 'Active', 'Center City East', 39.94733, -75.144028)
(3006, '40th & Spruce', '2015-04-23 00:00:00', 'Active', 'University City', 39.952202, -75.20311)
(3007, '11th & Pine, Kahn Park', '2015-04-23 00:00:00', 'Active', 'Washington Square West', 39.945171, -75.159927)
(3008, 'Temple University Station', '2015-04-23 00:00:00', 'Active', 'North Philadelphia', 39.98044755123083, -75.1506971812565)
(3009, '33rd & Market', '2015-04-23 00:00:00', 'Active', 'University City', 39.955761, -75.189819)
(3010, '15th & Spruce', '2015-04-23 00:00:00', 'Active', 'Rittenhouse Square', 39.947109, -75.166183)
(3011, '38th & Powelton', '2015-04-23 00:00:00', 'Active', 'University City', 39.95960020818546, -75.19664993981894)
(3012, '21st & Catharine', '2015-04-

<br>Check the number of rows

In [95]:
c.execute('''  
            SELECT COUNT(*) 
            FROM stations
            ''')

for row in c.fetchall():
    print (row)

(180,)


<br>The data has been stored in the 'bike_trip-db' database